In [1]:
import os, sys, time, shlex, subprocess

import psutil, GPUtil


def gpu_info() -> dict:
    gpus = GPUtil.getGPUs()
    if len(gpus) == 0: return {}
    gpu = gpus[0]

    # gpu.id, gpu.name, gpu.uuid
    return {
      "load_percent": round(gpu.load*100, 2),
      "memory_percent": round(gpu.memoryUsed/gpu.memoryTotal*100, 2),
      "temperature": gpu.temperature,
    }


class ProcX:
    cmd = []
    wd = ""
    n = 0
    process = None

    __cpu_interval = 1;

    def __init__(self, cmd: str):
        x = shlex.split(cmd)
        if len(x) < 2: return False
        self.cmd, self.wd = x, os.getcwd()

    def run(self) -> bool:
        if not self.process is None and self.process.is_running():
            print("!!! process is running")
            return False

        try:
            process = subprocess.Popen(self.cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            # process.kill()
        except:
            print("!!! command start failed 1:", sys.exc_info()[0])
            return False

        print(">>> ppid:", process.pid)
        process = psutil.Process(process.pid)
        for i in range(10):
            children = process.children()
            if len(children) == 0:
                print("... wait 500 millisecond")
                time.sleep(0.5)
                continue

            self.process = children[0]
            print(">>> pid:", self.process.pid)
            break

        if self.process is None:
            print("!!! command start failed 4:", sys.exc_info()[0])
            return False

        self.n = 1
        return True

    def is_running(self) -> bool:
        if self.process is None: return False

        return self.process.is_running()

    def execute(self, action: str) -> bool:
        if self.process is None: return False

        if not self.process.is_running(): return False
        status = self.process.status()

        if status != "running" and action in ["kill", "restart", "stop"]:
            return False
        elif status != "stopped" and action == "resume":
            return False

        if action == "kill":
            self.process.kill()
        elif action == "restart":
            self.process.kill()
            self.run()
        elif action == "stop":
            self.process.suspend()
        elif action == "resume":
            self.process.resume()
        else:
            print("!!! unkonwn action:", action)
            return False

        return True

    def status(self) -> dict:
        if self.process is None: return {}

        if not self.process.is_running(): return {"status": "terminated"}

        try:
            cpu_percent = round(self.process.cpu_percent(self.__cpu_interval)/psutil.cpu_count(), 2)
        except psutil.NoSuchProcess:
            return {"status": "terminated"}

        return {
            "staus": self.process.status(),
            "memory_percent": round(self.process.memory_percent(), 2),
            "cpu_percent": cpu_percent,
            "system_gpu_info": gpu_info(),
        }

In [2]:
os.getcwd()
os.chdir("D:\ShiBanQiao3-24")

cmd = "ShiBanQiao.exe -AudioMixer -PixelStreamingIP=47.117.115.165 -PixelStreamingPort=8204 -RenderOffScreen"
px = ProcX(cmd)

ok = px.run()
print(ok)

>>> ppid: 10836
... wait 500 millisecond
>>> pid: 11896
True


In [13]:
# px.process
px.isRunning()
px.status()

{'staus': 'running',
 'memory_percent': 15.33,
 'cpu_percent': 33.26,
 'system_gpu_info': {'load_percent': 87.0,
  'memory_percent': 84.18,
  'temperature': 76.0}}

In [14]:
px.execute("stop")
px.status()

{'staus': 'stopped',
 'memory_percent': 20.87,
 'cpu_percent': 0.0,
 'system_gpu_info': {'load_percent': 92.0,
  'memory_percent': 99.32,
  'temperature': 76.0}}

In [5]:
px.status()

{'staus': 'stopped',
 'memory_percent': 9.58,
 'cpu_percent': 0.0,
 'system_gpu_info': {'load_percent': 0.0,
  'memory_percent': 83.86,
  'temperature': 56.0}}

In [3]:
px.execute("resume")
px.status()

{'staus': 'running',
 'memory_percent': 22.33,
 'cpu_percent': 24.61,
 'system_gpu_info': {'load_percent': 92.0,
  'memory_percent': 89.58,
  'temperature': 77.0}}

In [4]:
# px.terminate()
px.execute("kill")
px.status()

{'status': 'terminated'}

In [8]:
px.status()

{'status': 'terminated'}

In [9]:
px.isRunning()

False

In [10]:
px.run()

>>> ppid: 6276
... wait 500 millisecond
>>> pid: 5176


True